In [434]:
%run gw_exp.ipynb
# import functions from gw_exp.ipynb

92


In [435]:
# ogw_exp computes the open gromov witten invariant of (CP^n, RP^n)
# Input:
# n - dimension of the real projective space RP^n
# A - list of cohomology classes represented by integers, ranging from 1...n, and -1 representing \Gamma_{square}
# b - beta, a non-negative integer
# k - number of boundary marked points, a non-negative integer
def ogw_invariant(n, b, k, A):
    l = len(A)

    A.sort()    # asceding order
    #A.reverse() # to descending order

    for c_d in A:
        if c_d < 0:
            print("negative classes")
        if c_d > n:
            return 0

    # wall-crossing theorem (Elad)
    if l > 0 and A[0] == 'a':
        A.pop(0) # remove the last element
        return -ogw_invariant(n, A, b, k+1)

    #degree axiom (Elad)
    if 2*sum(A) != (n - 3 + (n+1)*b + k + 2*l - k*n):  
        # The maslov index is twice the chern class n+1 for holomorphic spheres. 
        # Using the invoulotion we can deduce that its half of this for disks thus mu(b)=b*(n+1) 
        # p 595 mcduff 
        return 0

    # zero axiom
    if b == 0:
        if k == 1 and l == 1 and A[0] == 0:
            return -1
        else:
            return 0

    # unit axiom (May)
    # handle all beta = 0 cases
    if l > 0 and A[0] == 0:
        if b == 0 and k == 1 and l == 1:
            return -1
        else:
            return 0

    # divisor axiom 
    if l > 0 and A[0] == 1:
        A.pop(0)
        return (b/2)*ogw_invariant(n, b, k, A)

    # handle beta = 1, l <= 1 cases
    if b == 1 and k == 0 and A == [n] and is_even(n+3):
        return (-1)^((n+3)/2)

    if b == 1 and k == 1 and is_even(n+1) and A == [(n+1)/2]:
        return 0

    if b == 1 and k == 2 and A == []:
        return 2

    if l >= 2:
        result = ogw_invariant(n, b, k, [A[0]-1, A[1] + 1] + A[2:])
        if len(A) == 2:
            return result

        I = bipart(A[2:]) #bipart([A[i] for i in range(2, l)])
        print("theorem 10 (a)")
        print("b: ", b, ", k: ", k)
        print("original A: ", A)

        for t in I:
            I1 = t[0]
            I2 = t[1]

            # bigger range causes infinite loop
            for b1 in range(0, b+1):
                print("b1: ", b1)
                b_ = 0
                if (b - b1) % 2 == 0:
                    b_ = (b - b1) / 2
                else:
                    continue

                for i in range(0, n+1):
                    print("i: ", i)
                    print("ogw1: ", [n-i, A[0]-1] + I2)
                    print("ogw2: ", [n-i, A[1]] + I2)
                    result += (-1)^((n+1)*b_/2) * (gw_invariant(n, b_, [1, A[1], i] + I1) \
                        * ogw_invariant(n, b1, k, [n-i, A[0]-1] + I2) \
                            - gw_invariant(n, b_, [1, A[0] - 1, i] + I1) * \
                                ogw_invariant(n, b1, k, [n-i, A[1]] + I2))

        for t in I:
            I1 = t[0]
            I2 = t[1]
            
            # changing range has no effect on values
            for b1 in range(1, b):
                b2 = b - b1

                # smaller range cause incorrect value
                for k2 in range(0, k+1):
                    k1 = k - k2
                    #print("loop2")
                    result += binomial(k, k1)*(ogw_invariant(n, b1, k1, [1, A[0]-1] + I1) \
                        * ogw_invariant(n, b2, k2 + 1, [A[1]] + I2) \
                            - ogw_invariant(n, b1, k1, [1, A[1]] + I1) \
                                * ogw_invariant(n, b2, k2 + 1, [A[0] - 1] + I2))

        return result

    if k >= 2:
        print("theorem 10 (b) ")
        print("b: ", b, " , k: ", k, ", A: ", A)
        result = 0

        I = bipart(A)

        for t in I:
            I1 = t[0]
            I2 = t[1]

            # not much effect for n = 7
            for b1 in range(1, b+1):
                b_ = 0
                if (b + 1 - b1) % 2 == 0:
                    b_ = (b + 1 - b1) / 2
                else:
                    break

                for i in range(0, n+1):
                    #print("loop3")
                    result += 2*(-1)^((n+1)*b_/2)*gw_invariant(n, b_, [1, n, i] + I1) \
                        * ogw_invariant(n, b1, k-1, [n-i] + I2)

        for t in I:
            I1 = t[0]
            I2 = t[1]
            
            for b1 in range(2, b+1):
                b2 = b + 1 - b1

                for k1 in range(0, (k - 2)+1):
                    k2 = k - 2 - k1
                    #print("loop4")
                    result -= 2*binomial(k, k1)*ogw_invariant(n, b1, k1, [1, n] + I1)\
                        * ogw_invariant(n, b2, k2+2, I2)

            for b1 in range(1, b+1):
                b2 = b + 1 - b1

                for k1 in range(0, (k - 2)+1):
                    k2 = k - 2 - k1
                    #print("loop5")
                    result += 2*binomial(k, k1)*ogw_invariant(n, b1, k1 + 1, [1] + I1) \
                        *ogw_invariant(n, b2, k2+1, [n]+I2)

        return result * (-1)^((n+3)/2)

    print("unexpected situation")
    print("n: ", n, ", b: ", b, ", k: ", k, " A: ", A)
    return 0
"""
try:
    print("should be -2")
    print(ogw_invariant(3, 3, 6, []))
except RecursionError:
    print("infinite loop")

try:
    print("should be 2")
    print(ogw_invariant(5, 5, 8, []))
except RecursionError:
    print("infinite loop")

try:
    print("should be 1/8")
    print(ogw_invariant(5, 1, 0, [2, 2, 2, 2]))
except RecursionError:
    print("infinite loop")

try:
    print("should be 1/2")
    print(ogw_invariant(5, 1, 0, [2, 4]))
except RecursionError:
    print("infinite loop")


try:
    print("should be 3/8")
    print(ogw_invariant(5, 3, 0, [2]*1+[4]*3))
except RecursionError:
    print("infinite loop")

try:
    print("should be -11/32")
    print(ogw_invariant(5, 3, 0, [2]*4+[4]*2))
except RecursionError:
    print("infinite loop")

try:
    print("should be -255/32") # missing 1/4 somewhere
    print(ogw_invariant(5, 3, 0, [2]*7+[4]*1))
except RecursionError:
    print("infinite loop")

try:
    print("should be 601224741985/32768")
    #print(ogw_invariant(5, 3, 0, [2]*10))
except RecursionError:
    print("infinite loop")



try:
    print("should be -1/32")
    print(ogw_invariant(7, 1, 0, [2]*6))
except RecursionError:
    print("infinite loop")

try:
    print("should be -1/2")
    print(ogw_invariant(7, 1, 0, [2, 6]))
except RecursionError:
    print("infinite loop")


try:
    print("should be -11/32")
    print(ogw_invariant(7, 3, 0, [2]*4 + [6]*2))
except RecursionError:
    print("infinite loop")

try:
    print("should be -23229/512")
    print(ogw_invariant(7, 3, 0, [2]*9 + [6]*1))
except RecursionError:
    print("infinite loop")

try:
    print("should be -338633/512")
    #rint(ogw_invariant(7, 5, 0, [2]*7 + [6]*3))
except RecursionError:
    print("infinite loop")
"""
try:
    print("should be 1/2")
    print(ogw_invariant(5, 1, 0, [2, 4]))
except RecursionError:
    print("infinite loop")


try:
    print("should be -11/32")
    print(ogw_invariant(5, 3, 0, [2]*4+[4]*2))
except RecursionError:
    print("infinite loop")

try:
    print("should be -255/32") # missing 1/4 somewhere
    # l1 = 7, l2 = 1
    print(ogw_invariant(5, 3, 0, [2]*7 + [4]*1))
except RecursionError as re:
    print("infinite loop")
    #traceback.print_tb(re.__traceback__)



should be 1/2
1/2


'\ntry:\n    print("should be -11/32")\n    print(ogw_invariant(5, 3, 0, [2]*4+[4]*2))\nexcept RecursionError:\n    print("infinite loop")\n\ntry:\n    print("should be -255/32") # missing 1/4 somewhere\n    # l1 = 7, l2 = 1\n    print(ogw_invariant(5, 3, 0, [2]*7 + [4]*1))\nexcept RecursionError as re:\n    print("infinite loop")\n    #traceback.print_tb(re.__traceback__)\n'

In [436]:
def get_l1(n, b, k, l2, A1, A2):
    m = n - 3 + b*(n+1) + k + 2*l2 - k*n - 2*l2 * A2
    if (m % (2*(A1 - 1))) == 0:
        return m / (2*(A1 - 1))
    else:
        return -1

print(get_l1(5, 1, 0, 0, 2, 4))
print(get_l1(5, 1, 0, 1, 2, 4))
print(get_l1(5, 1, 0, 2, 2, 4))
print(get_l1(5, 1, 0, 3, 2, 4))

print(" ")
print(get_l1(5, 3, 0, 0, 2, 4))
print(get_l1(5, 3, 0, 3, 2, 4))
print(get_l1(5, 3, 0, 2, 2, 4))
print(get_l1(5, 3, 0, 1, 2, 4))



print("  ")


print(get_l1(7, 1, 0, 0, 2, 6))
print(get_l1(7, 1, 0, 1, 2, 6))

print(get_l1(7, 3, 0, 2, 2, 6))

print(get_l1(7, 3, 0, 1, 2, 6))

print(get_l1(7, 5, 0, 3, 2, 6))


4
1
-2
-5
 
10
1
4
7
  
6
1
4
9
7


In [437]:
print(binomial(-2, 4))

5
